<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Supply Chain Visibility
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>
Businesses have a long recognized the pivotal role of supply chain in their success. Supply chain visibility is the ability to view or track goods and materials as it moves through the supply chain. Businesses know that this creates and sustains a competitive advantage. Below are some of the critical capabilities of Supply Chain Visibility that creates a competitive advantage:
<ul style = 'font-size:16px;font-family:Arial'>
<li>Quickly adapt to rapidly changing market conditions</li>
    <li>Use accurate insights to provide better service, enhancing customer satisfaction</li>
    <li>Understand the levels of risk of my material or goods not arriving when customers need them leading to Customer Service level failure, or arriving to early leading to storage and cost challenges</li>
    <li>Use data to drive innovation and continuous improvement</li>
    <li>Leverage data effectively and maximize investments in existing solutions</li>
    <li>Streamline operations, reduce redundancies, and optimize workflows</li>
    <li>Drive down the cost of goods sold and improve profitability</li>
    <li>Reduce inventory carrying costs while maintaining required service levels</li>
    </ul>
<p style = 'font-size:16px;font-family:Arial'>Despite knowing the importance and advantages of visibility, business fail to implement and answer the basic questions like <ul style = 'font-size:16px;font-family:Arial'><li>Where is my inventory, what is in-transit and when will it arrive?</li><li>
    What are my current lead times by vendor, item and receipt location?</li></ul>
    <p style = 'font-size:16px;font-family:Arial'>Modern supply chains are comprised of highly specialized functional silos. This is a demonstration of managing complexity across multiple source systems that at any point in time have a different view of the millions of transactions they process. </p> 
<img src="images/silo.png">    
<p style = 'font-size:16px;font-family:Arial'>
 Supply chain visibility requires a data foundation to be semantically consistent. Teradata's supply chain visibility solution accelerator comes into play because the Visibility Data Foundation is an open platform that is extensible and brings together all of the relevant pieces of your supply chain. It connects the source data via a consistent semantic model, enriches it and then it drives collaboration across the chain by 
automating analytics and insights by leveraging the in-Db capabilities of Clearscape Analytics.<br><br>The following diagram shows a manufacturing flavor of supply chain, but remember these challenges are equally true for all supply chains such as retail, its just that the specific terminology changes</p>   
<img src="images/scv.png">  

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>1. Import python packages, connect to Vantage and explore the dataset</b></p>

In [ ]:
#import libraries
import geopandas as gpd
import matplotlib.pyplot as plt 
import pandas as pd

import getpass
from teradataml import *
import plotly.express as px
import plotly.graph_objects as go

import numpy as np

display.max_rows = 5 

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then <b>use down arrow</b> to go to next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Supply_Chain_Visibility.ipynb;' UPDATE FOR SESSION; ''')


<p style = 'font-size:18px;font-family:Arial'> <b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. To leverage the temporal capabilities of CLearScape Analytics, will will import the data into temporal tables on local storage. </p>

In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_SupplyChain_local');" 
# takes about 1 min 40 seconds, estimated space: 1.5 MB

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>2. Architecture Diagram</b></p> 
<img src="images/sc_visibility.png">  

<p style = 'font-size:16px;font-family:Arial'> As we can see there are various sources of data in a supply chain which will send data at different time intervals. To make informed decisions from the existing and incoming data; it is imperative that the should be in correct form.<br>using Vantage's inDb capabilities <b>Data Quality</b> and <b>Rule based checks</b> can be applied to incoming data pipeline before the reports or analysis can be done on them.  

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>3. Source tables</b></p> 

<p style = 'font-size:16px;font-family:Arial'>Let us take a look at the data in the source tables we have.<br>For our demo we have source information coming from customer, materials warehouse and transportation <u>O</u>rder <u>M</u>anagement <u>S</u>ystems (OMS). Each of these sources will update the information independent of each other and will be loaded in the EDW at separate times.<br><b>Customer</b><br>The data from the customer system have information about the order placed like when it was placed, requested arrival time, city where the order should deliver etc. The Customer system runs a batch load to EDW every 8 hours (@ 2:00, 10:00, 18:00) hence the updates in the source system happening at 7.00am will be reflected at 10.00am then the load to data warehouse is complete. This is usually reflected in the EDW start and end times (dwh_startts and dwh_endts) in the Customer_OMS table.<br><b>Transportation</b><br>Similarly the transportation system loads data every 4hours 1:00, 5:00, 9:00, 13:00, 17:00, 23:00. and has information regarding the movement of the trucks etc. <br><b>Material Warehouse</b><br>The materials warehouse system has information regarding the order fulfilment and planing and has point in time updates i.e the updates from the source system are reflected at the same time in EDW.<br>In the actual projects business there are hundreds of files come for processing in EDW and it is controlled by date control tables which keeps track of the timestamps of each source system batch
</p>


In [ ]:
tdf_customer_oms = DataFrame(in_schema("DEMO_SupplyChain","Customer_OMS"))
tdf_customer_oms

In [ ]:
tdf_customer_oms.tdtypes

<p style = 'font-size:16px;font-family:Arial'>In the data above we can see that dwh_startts and dwh_endts are the start and end time of when the data was loaded in the warehouse. We can create temporal table based on the start and end times. The <b>Temporal database</b> is time aware. The difference between a temporal database and a conventional database is that a temporal database maintains data with respect to time and allows time-based reasoning, whereas a conventional database captures only a current snapshot of reality. <br><br> There are three types of temporal tables:
<ol><li style = 'font-size:16px;font-family:Arial'> TRANSACTIONTIME represents the time period where a fact was known to the database and is automatically maintained. </li>
    <li style = 'font-size:16px;font-family:Arial'> VALIDTIME represents the time period when something in the real world was in effect. For example: the period where a shipment was in transit.</li>
    <li style = 'font-size:16px;font-family:Arial'> Bitemporal tables have both TRANSACTIONTIME period and VALIDTIME period. </li></ol>
<p style = 'font-size:16px;font-family:Arial'>The table below is a TRANSACTIONTIME table. 

In [ ]:
%%capture
query1 = '''
CREATE MULTISET TABLE DEMO_User.Customer_OMS_Temporal 
     (
      Order_Id INTEGER,
      Order_Ts TIMESTAMP(0),
      Order_Status VARCHAR(30) CHARACTER SET LATIN NOT CASESPECIFIC,
      Requested_Arrival_Ts TIMESTAMP(0),
      Actual_Arrival_Ts TIMESTAMP(0),
      Customer_Id VARCHAR(20) CHARACTER SET LATIN NOT CASESPECIFIC,
      Customer_City VARCHAR(20) CHARACTER SET LATIN NOT CASESPECIFIC,
      Quantity INTEGER,
      Event_Ts TIMESTAMP(0),
      Dwh_Duration PERIOD(TIMESTAMP(6) WITH TIME ZONE) NOT NULL AS TRANSACTIONTIME
  )
PRIMARY INDEX ( Order_Id );
'''
query2='''GRANT NONTEMPORAL on DEMO_User.Customer_OMS_Temporal to demo_user; '''

execute_sql(query1)
execute_sql(query2)

<p style = 'font-size:16px;font-family:Arial'>In a production application inserting data into a TRANSACTIONTIME table, the insert statement would take the form of:
 <br>&emsp;insert into DEMO_User.Customer_OMS_TEMPORAL (field list excluding the TRANSACTIONTIME column)
 <br>&emsp;select ( field list excluding the TRANSACTIONTIME column);
 <br>Vantage will automatically update and existing row's period ending with the current timestamp and create a new row with period beginning as the current timestamp and the ending timestamp far into the future.
    <br><br>
 if we want to load the history data in these tables, we have to apply NONTEMPORAL clause to allow initial setting of the TRANSACTIONTIME column.

In [ ]:
%%capture
query = '''
NONTEMPORAL
insert into  DEMO_User.Customer_OMS_Temporal 
select 
      Order_Id,
      Order_Ts,
      Order_Status,
      Requested_Arrival_Ts,
      Actual_Arrival_Ts,
      Customer_Id ,
      Customer_City,
      Quantity ,
      Event_Ts,
      CASE WHEN cast(Dwh_Endts as timestamp(6) with time zone) = '9999-12-31 00:00:00.000000-05:00' then 
      PERIOD(cast(Dwh_Startts as timestamp(6) with time zone), UNTIL_CHANGED)
      else PERIOD(cast(Dwh_Startts as timestamp(6) with time zone),cast(dwh_endts as timestamp(6) with time zone))
      end
from  DEMO_SupplyChain.Customer_OMS
;
'''

execute_sql(query)

In [ ]:
tdf_customer_oms_t = DataFrame(in_schema("DEMO_User","Customer_OMS_Temporal"))
tdf_customer_oms_t

<p style = 'font-size:16px;font-family:Arial'>From the above output we can see that all of the rows are "Shipment Received" which is typically the last event for a transaction.<br>
  <ul style = 'font-size:16px;font-family:Arial'>
      <li>The beginning of the transaction-time period is the time when the database became aware of a row,
        when the row was first recorded in the database. This is when the row was added to a table.</li>
      <li>The end of a transaction time period reflects when the fact was superseded by an update to the row,
or when the row was deleted from the database. Rows containing information that is currently in effect
        have transaction-time periods with indefinite ending bounds, represented as UNTIL_CLOSED.</li>
        </ul>
 <p style = 'font-size:16px;font-family:Arial'>Let us focus on one order (13540) to understand what temporal is doing. The dataframe output above truncated the Dwh_Duration column, so we will use SQL to get it to display the whole value.</p>

In [ ]:
tdf_c1= DataFrame.from_query('''select top 5 order_id, order_ts, order_status, requested_arrival_ts, actual_arrival_ts, 
Customer_id, customer_city, quantity, event_ts, cast(dwh_duration as varchar(60)) dwh_duration 
from demo_user.customer_oms_temporal where order_id = 13540;''')
tdf_c1

<p style = 'font-size:16px;font-family:Arial'>From the above results we can see that due to temporal table, simple select will always display the active records only. The event timestamp for shipment received was at 18:14:00 but the Event Timestamp (when the data was posted to the EDW) was at 2:00 the next day. The end of the period (dwh_duration ... data warehouse duration) was the timestamp associated with "UNTIL_CLOSED".
    <br><br>For displaying all the records active and closed we need to provide 'NONSEQUENCED TRANSACTIONTIME' prefix in the query.</p>

In [ ]:
tdf_c2= DataFrame.from_query('''NONSEQUENCED TRANSACTIONTIME select top 5 order_id, order_ts, order_status, requested_arrival_ts, actual_arrival_ts, 
Customer_id, customer_city, quantity, event_ts, cast(dwh_duration as varchar(60)) dwh_duration 
from demo_user.customer_oms_temporal where order_id = 13540 order by event_ts desc;''')
tdf_c2.sort('Event_Ts')

<p style = 'font-size:16px;font-family:Arial'>Now we can see all the records active or closed for the order_id 13540. We can see that the order information arrived in warehouse on 2023-04-06 and it's status is changed three times. 
<br><br>
Lets add an update to show Temporal working: </p>

In [ ]:
%%capture
query = '''
Update DEMO_User.Customer_OMS_Temporal set
      Order_Status = 'Shipment Return',
      Event_Ts = '2023-04-12 11:30:00'
      where Order_Id = 13540
 ;
'''

execute_sql(query)   

In [ ]:
# first showing the default view of the current transaction status

tdf_c1= DataFrame.from_query('''select top 5 order_id, order_ts, order_status, requested_arrival_ts, actual_arrival_ts, 
Customer_id, customer_city, quantity, event_ts, cast(dwh_duration as varchar(60)) dwh_duration 
from demo_user.customer_oms_temporal where order_id = 13540;''')
tdf_c1

In [ ]:
# and showing the complete history of the transaction

tdf_c2= DataFrame.from_query('''NONSEQUENCED TRANSACTIONTIME select top 5 order_id, order_ts, order_status, requested_arrival_ts, actual_arrival_ts, 
Customer_id, customer_city, quantity, event_ts, cast(dwh_duration as varchar(60)) dwh_duration 
from demo_user.customer_oms_temporal where order_id = 13540 order by event_ts desc;''')
tdf_c2.sort('Event_Ts')

 <p style = 'font-size:16px;font-family:Arial'>You could change the update statement above to create another event (like 'Return Accepted') and then rerun the subsequent queries to see the impact.
    <br><br>Before proceeding further, let's create temporal tables for Warehouse_OMS and Transportation_OMS.</p>

In [ ]:
%%capture
# Query to create the warehouse order management system table
query1 = '''
CREATE MULTISET TABLE DEMO_User.Warehouse_OMS_Temporal 
     (
      Ship_Id INTEGER,
      Shipping_Ts TIMESTAMP(0),
      Shipment_Status VARCHAR(20) CHARACTER SET LATIN NOT CASESPECIFIC,
      Ship_From VARCHAR(20) CHARACTER SET LATIN NOT CASESPECIFIC,
      Ship_To VARCHAR(20) CHARACTER SET LATIN NOT CASESPECIFIC,
      Order_Id INTEGER,
      Customer_Id VARCHAR(20) CHARACTER SET LATIN NOT CASESPECIFIC,
      Order_Ts TIMESTAMP(0),
      Quantity INTEGER,
      Expected_Delivery_Ts TIMESTAMP(0),
      Actual_Delivery_Ts TIMESTAMP(0),
      Event_Ts TIMESTAMP(0),
      Dwh_Duration PERIOD(TIMESTAMP(6) WITH TIME ZONE) NOT NULL AS TRANSACTIONTIME
     )
PRIMARY INDEX ( Ship_Id );
'''

query2='''GRANT NONTEMPORAL on DEMO_User.Warehouse_OMS_Temporal to demo_user; '''

# Query to create the transportation order management system table
query3 = '''
CREATE MULTISET TABLE DEMO_User.Transportation_OMS_Temporal 
     (
      Ship_Id INTEGER,
      Order_Id INTEGER,
      Status VARCHAR(20) CHARACTER SET LATIN NOT CASESPECIFIC,
      Origin VARCHAR(20) CHARACTER SET LATIN NOT CASESPECIFIC,
      Destination VARCHAR(20) CHARACTER SET LATIN NOT CASESPECIFIC,
      Event_Ts TIMESTAMP(0),
      Dwh_Duration PERIOD(TIMESTAMP(6) WITH TIME ZONE) NOT NULL AS TRANSACTIONTIME
   )
PRIMARY INDEX ( Ship_Id );
'''

query4='''GRANT NONTEMPORAL on DEMO_User.Transportation_OMS_Temporal to demo_user; '''


# Query to populate the warehouse order management system temporal table
query5 = '''
NONTEMPORAL
insert into  DEMO_User.Warehouse_OMS_Temporal 
select 
      Ship_Id,
Shipping_Ts,
Shipment_Status,
Ship_From,
Ship_To,
Order_Id,
Customer_Id,
Order_Ts,
Quantity,
Expected_Delivery_Ts,
Actual_Delivery_Ts,
Event_Ts,
CASE WHEN cast(Dwh_Endts as timestamp(6) with time zone) = '9999-12-31 00:00:00.000000-05:00' then 
     PERIOD(cast(Dwh_Startts as timestamp(6) with time zone), UNTIL_CHANGED)
     else PERIOD(cast(Dwh_Startts as timestamp(6) with time zone),cast(dwh_endts as timestamp(6) with time zone))
end
from  DEMO_SupplyChain.Warehouse_OMS
; 
'''

# Query to populate the transportation order management system temporal table
query6 = '''
NONTEMPORAL
insert into  DEMO_User.Transportation_OMS_Temporal 
select 
Ship_Id,
Order_Id,
Status,
Origin,
Destination,
Event_Ts,
CASE WHEN cast(Dwh_Endts as timestamp(6) with time zone) = '9999-12-31 00:00:00.000000-05:00' then 
     PERIOD(cast(Dwh_Startts as timestamp(6) with time zone), UNTIL_CHANGED)
     else PERIOD(cast(Dwh_Startts as timestamp(6) with time zone),cast(dwh_endts as timestamp(6) with time zone))
end
from  DEMO_SupplyChain.Transportation_OMS
;  
'''
execute_sql(query1);
execute_sql(query2);
execute_sql(query3);
execute_sql(query4);
execute_sql(query5);
execute_sql(query6);

 <p style = 'font-size:16px;font-family:Arial'> Now let us check the Transportation_Oms and Warehouse_Oms records for the order id 13540.

In [ ]:
tdf_transport_oms = DataFrame(in_schema("DEMO_User","Transportation_OMS_Temporal"))
tdf_transport_oms.loc[(tdf_transport_oms.Order_Id == 13540)]

In [ ]:
tdf_t= DataFrame.from_query('''NONSEQUENCED TRANSACTIONTIME 
select  Ship_Id, Order_Id, Status, Origin, Destination, Event_Ts, cast(Dwh_Duration as varchar(60)) Dwh_Duration 
from DEMO_User.Transportation_OMS_Temporal where Order_Id = 13540;''')
tdf_t.sort('Event_Ts')

In [ ]:
tdf_warehouse_oms = DataFrame(in_schema("DEMO_User","Warehouse_OMS_Temporal"))
tdf_warehouse_oms.loc[(tdf_warehouse_oms.Order_Id == 13540)]

In [ ]:
tdf_w= DataFrame.from_query('''NONSEQUENCED TRANSACTIONTIME 
select Ship_Id, Shipping_Ts, Shipment_Status, Ship_From, Ship_To, Order_Id, Customer_Id, Order_Ts, 
Quantity, Expected_Delivery_Ts, Actual_Delivery_Ts, Event_Ts, cast(Dwh_Duration as varchar(60)) Dwh_Duration 
from DEMO_User.Warehouse_OMS_Temporal where Order_Id = 13540;''')
tdf_w.sort('Event_Ts')

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>4. What is the status of my shipment?</b></p> 

<p style = 'font-size:16px;font-family:Arial'>As we have seen before due to complexity and silos of each of the source system in large supply chains, business struggle to get the basic answers of question.

<img src="images/sc_problem.png">  

<p style = 'font-size:16px;font-family:Arial'>Different systems can give different views due to latency of refresh of the data from the source system so which one is correct? In our example we know the Customer system only updates every 8 hours, last update was 6 hours ago so its likely out of date, warehouse system is real time and shows the order has arrived – logic is the warehouse system is probably right. 
<img src="images/timeline.png">     
<br>The above image shows the overall timeline of status of order 13540 between all the 3 systems we have. Please note that the datawarehouse update customer and transportation system in batch so if multiple updates happen in batch window only the latest will be inserted in the EDW.    
    <br><br>Consider a scenario where customer calls asking ‘where is my order?’ – EDW insight response could be, it was delivered 4 hours ago, customer systems last updated 6 hours ago so you can’t see it yet, your system update cycle is 8 hours so please recheck your system in 2hrs time. If after 3 hrs from now Customer system has updated and order still isn’t showing, then action is to investigate because something has gone wrong (maybe delivered to wrong loading dock / good damaged on delivery so not receipted in etc).<br>Prefix TRANSACTIONTIME AS OF 'timestamp' in the temporal table will give the valid record as of the transaction time specified.<br><br> For our example let us consider we want to check what is the status of my order at 2023-04-11 at 5.10Pm.  </p>

In [ ]:
tdf_status= DataFrame.from_query('''TRANSACTIONTIME AS OF TIMESTAMP'2023-04-11 17:10:00.000000+00:00' 
select  w.order_id as order_id, w.shipment_status  as "status_warehouse_oms", begin(w.dwh_duration)  w_oms_last_update_ts ,
 c.order_status as "status_customer_oms", begin(c.dwh_duration) as customer_oms_last_update_ts,
 t.status as "status_transport_oms", begin(t.dwh_duration) as transportation_oms_last_update_ts
from demo_user.warehouse_oms_temporal w
join demo_user.customer_oms_temporal c
on w.order_id = c.order_id
join demo_user.transportation_oms_temporal t
on w.order_id = t.order_id
where w.order_id = 13540;''')
tdf_status

<p style = 'font-size:16px;font-family:Arial'>Here we can see the shipment status in all the three systems at time 11th April 2023, 5:10PM. With the help of temporal datatype we don't have to add any date conditions in query to get the active record from each system at the specified time, the database does this automatically. 

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>5. Customer and Warehouse locations</b></p> 

<p style = 'font-size:16px;font-family:Arial'>Once we have all the data loaded in the warehouse, we can also find the other information easily like the customer and warehouse locations.

In [ ]:
tdf_cust = DataFrame(in_schema("DEMO_SupplyChain","Customer"))
tdf_cust

In [ ]:
tdf_cust.shape

<p style = 'font-size:16px;font-family:Arial'>We have 25 customers in our demo.

In [ ]:
tdf_ware = DataFrame(in_schema("DEMO_SupplyChain","Warehouse"))
tdf_ware

In [ ]:
tdf_ware.shape

<p style = 'font-size:16px;font-family:Arial'>We have 5 warehouse in our demo. Let us plot them on the map.

In [ ]:
df_cust=tdf_cust.to_pandas().reset_index()
df_ware= tdf_ware.to_pandas().reset_index()

fig1 = px.scatter_mapbox(df_cust, title='Customer Locations',lat="Lat", lon="Lng", hover_name="City", 
                        hover_data=["Id"],
                        color_discrete_sequence=["blue"], zoom=3, height=500) 
fig1.add_trace(go.Scattermapbox(
    name = "Warehouse Location",
    mode = "markers",
    lat = df_ware.Lat.tolist(),
    lon = df_ware.Lng.tolist(),
    text=df_ware.Id ,
    hoverinfo='text',
    marker=go.scattermapbox.Marker(
            size=10,
            color='rgb(0,255, 0)',
            opacity=0.7
    )  
))
fig1.update_layout(mapbox_style="open-street-map" , #"stamen-terrain",
                  mapbox_zoom=3,
                  title = 'Customer and Warehouse Locations',
                  title_y=1,
                  margin={"r":0,"t":0,"l":0,"b":0}
                 )    
    
fig1.show()  

<p style = 'font-size:16px;font-family:Arial'>From the map above we can see the locations of customers and warehouse. Now let us plot a map to show the standard warehouse that is assigned to each customer based on its location.

In [ ]:
query = '''
select c.city||'_'||c."name"||'_'||c.id as id,
upper(Std_Source) as warehouse_id,
c.Lng,
c.Lat, 
w.Lng as ware_lng, 
w.Lat as ware_lat
from DEMO_SupplyChain.Customer c
join DEMO_SupplyChain.warehouse w
on c.Std_Source = w.id
;
'''

df_std_ware = DataFrame.from_query(query)
df_std_ware.sort('warehouse_id')


In [ ]:
# define figure
df_std_ware=df_std_ware.to_pandas()
fig = go.Figure()

for i in range(1, df_std_ware.shape[0]):
    df_path = df_std_ware[i:i+1]
    lats = df_path.Lat.tolist()+df_path.ware_lat.tolist()
    lons = df_path.Lng.tolist()+df_path.ware_lng.tolist()
    
    fig.add_trace(go.Scattermapbox(
        name = str(df_path.id.values[0]),
        mode = "markers+lines",
        lat = lats,
        lon = lons,
        hoverinfo='text',
        hovertemplate= ['<b>City_Name_Id:</b>:' + str(df_path.iloc[i, 0]) + '<br><i>Warehouse Id</i>:' + str(df_path.iloc[i, 1]) for i in range(df_path.shape[0])], 
        marker_color='blue' ,
        marker=dict(size = 6) ,
        line_color = 'cadetblue' ,
        showlegend=False
    ))
fig.add_trace(go.Scattermapbox(
    name = "Warehouse Location",
    mode = "markers",
    lat = df_ware.Lat.tolist(),
    lon = df_ware.Lng.tolist(),
    text=df_ware.Id ,
    hoverinfo='text',
    marker=go.scattermapbox.Marker(
            size=10,
            color='rgb(0,255, 0)',
            opacity=0.7
    )  
))

fig.update_layout(mapbox_style="open-street-map" , #"stamen-terrain",
                  mapbox_zoom=3,
                  mapbox_center_lon=-95.7129,
                  mapbox_center_lat=37.0902,
                  title = 'Customer Locations serviced by Warehouse',
                  title_y=1,
                  margin={"r":0,"t":0,"l":0,"b":0}
                 )        
       
fig.show()

<p style = 'font-size:16px;font-family:Arial'>Now let us assume that WHSE202 is not servicable due to some reason. In this situation we need to assign another warehouse to the customers which were serviced by warehouse WHSE202. <br>If we consider distance between the customer location and warehouse location as the criteria for reassigning new warehouse we can do that very efficiently using the geometry functions available in Vantage. In this case we will use <b>ST_SphericalDistance</b> which calculates spherical distance between two spherical coordinates on the planet using the Haversine Formula(takes in account of Earth's curvature while calculating distance). 

In [ ]:

query = '''
select id,warehouse_id,Lng,Lat,new_warehouse_id,Distance_Meters,
ware_lng, 
ware_lat
from
(
select c.city||'_'||c."name"||'_'||c.id as id,
upper(Std_Source) as warehouse_id,
c.Lng,
c.Lat,
upper(w.Id) as new_warehouse_id,
w.Lng as ware_lng, 
w.Lat as ware_lat,
ROUND(new ST_Geometry('ST_Point', c.lng, c.lat).ST_SphericalDistance(new ST_Geometry('ST_Point', w.lng, w.lat)), 0) Distance_Meters
from DEMO_SupplyChain.Customer c 
join DEMO_SupplyChain.warehouse w 
 on 1=1
where c.Std_Source = 'WHSE202'
and w.Id <> 'WHSE202'
)x
QUALIFY ROW_NUMBER() OVER (partition by id,warehouse_id,Lng,Lat ORDER BY Distance_Meters) = 1;
'''

df_nw_ware = DataFrame.from_query(query)
df_nw_ware.sort('new_warehouse_id')
df_nw_ware=df_nw_ware.to_pandas()
df_nw_ware.head()

<p style = 'font-size:16px;font-family:Arial'>Let us plot all this information in the map for easier understanding.

In [ ]:
# define figure
df_ok_ware=df_std_ware[df_std_ware['warehouse_id'] != 'WHSE202'] 
df_no_ware=df_std_ware[df_std_ware['warehouse_id'] == 'WHSE202'] 
fig = go.Figure()

for i in range(0, df_ok_ware.shape[0]):
    df_path = df_ok_ware[i:i+1]
    lats = df_path.Lat.tolist()+df_path.ware_lat.tolist()
    lons = df_path.Lng.tolist()+df_path.ware_lng.tolist()
    
    fig.add_trace(go.Scattermapbox(
        name = str(df_path.id.values[0]),
        mode = "markers+lines",
        lat = lats,
        lon = lons,
        hoverinfo='text',
        hovertemplate= ['<b>City_Name_Id:</b>:' + str(df_path.iloc[i, 0]) + '<br><i>Warehouse Id</i>:' + str(df_path.iloc[i, 1]) for i in range(df_path.shape[0])], 
        marker_color='blue' ,
        marker=dict(size = 6) ,
        line_color = 'cadetblue' ,
        showlegend=False
    ))
for i in range(0, df_no_ware.shape[0]):
    df_path2 = df_no_ware[i:i+1]
    lats = df_path2.Lat.tolist()+df_path2.ware_lat.tolist()
    lons = df_path2.Lng.tolist()+df_path2.ware_lng.tolist()
    
    fig.add_trace(go.Scattermapbox(
        name = 'not in service',
        mode = "markers+lines",
        lat = lats,
        lon = lons,
        hoverinfo='text',
        hovertemplate= ['<b>City_Name_Id:</b>:' + str(df_path2.iloc[i, 0]) + '<br><i>Warehouse Id</i>:' + str(df_path2.iloc[i, 1]) for i in range(df_path2.shape[0])], 
        marker_color='blue' ,
        marker=dict(size = 6) ,
        line_color = 'grey' ,
    ))
for i in range(0, df_nw_ware.shape[0]):
    df_path3 = df_nw_ware[i:i+1]
    lats = df_path3.Lat.tolist()+df_path3.ware_lat.tolist()
    lons = df_path3.Lng.tolist()+df_path3.ware_lng.tolist()
    
    fig.add_trace(go.Scattermapbox(
        name = 'new warehouse'+ str(df_path3.id.values[0])[-9:],
        mode = "lines",
        lat = lats,
        lon = lons,
        hoverinfo='text',
        hovertemplate= ['<b>City_Name_Id:</b>:' + str(df_path3.iloc[i, 0]) + '<br><i>New Warehouse Id</i>:' + str(df_path3.iloc[i, 4]) + '<br><i>Distance</i>:' + str(df_path3.iloc[i, 5]) for i in range(df_path3.shape[0])], 
        line_color = 'purple' ,
    ))    
fig.add_trace(go.Scattermapbox(
    name = "Warehouse Location",
    mode = "markers",
    lat = df_ware.Lat.tolist(),
    lon = df_ware.Lng.tolist(),
    text=df_ware.Id ,
    hoverinfo='text',
    marker=go.scattermapbox.Marker(
            size=10,
            color='rgb(0,255, 0)',
            opacity=0.7
    )  
))    
fig.update_layout(mapbox_style="open-street-map" , #"stamen-terrain",
                  mapbox_zoom=3,
                  mapbox_center_lon=-95.7129,
                  mapbox_center_lat=37.0902,
                  title = 'Customer Locations serviced by New Warehouse',
                  title_y=1,
                  margin={"r":0,"t":0,"l":0,"b":0}
                 )        
       
fig.show()

<p style = 'font-size:16px;font-family:Arial'>In the map above the grey lines are depicting the path which are not serviced and purple line are depicting the paths from new assigned warehouse.

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>6. Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial'>In this demo we have touched upon a very basic problem that happens in supply chains and visibility at a whole EDW level can help in giving the exact version of events avoiding siloed views from each department. Having a single version of truth also helps us mitigating the unforeseen events happening in the supply chain. This can be further enhanced to give the other analytics like inventory levels at the material warehouse to future demand forecasting or by integrating the real-time locations of the trucks/transport to predict delays etc. 

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'> <b>7. Cleanup </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
tables = ["Customer_OMS_Temporal","Warehouse_OMS_Temporal","Transportation_OMS_Temporal"]

for t in tables:
        try:
            db_drop_table(table_name=t)
        except:
            pass

<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_SupplyChain');"        # Takes 5 seconds

In [ ]:
remove_context()

</h4>
<p style = 'font-size:20px;font-family:Arial'><b>Reference Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Information about Temporal datatype can be found at: <a href = 'https://docs.teradata.com/search/all?query=temporal&content-lang=en-US'>Temporal Datatype</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            © 2023 Teradata. All rights reserved.
        </div>
    </div>
</footer>